<a href="https://colab.research.google.com/github/Narwhalprime/vertex-ai-samples/blob/ready_to_go_notebook_fix/notebooks/community/pipelines/google_cloud_pipeline_components_ready_to_go_text_classification_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="在Colab中打开"/></a>

In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 顶点管道：Ready-to-go文本分类模型训练管道

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/pipelines/google_cloud_pipeline_components_ready_to_go_text_classification_pipeline.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在 Colab 中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/pipelines/google_cloud_pipeline_components_ready_to_go_text_classification_pipeline.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在 GitHub 上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/ai/platform/notebooks/deploy-notebook?download_url=https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/pipelines/google_cloud_pipeline_components_ready_to_go_text_classification_pipeline.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="顶点 AI logo">
      在顶点 AI 工作台中打开
    </a>
  </td>
</table>

## 概述

本演示展示了使用[Google Cloud Pipeline Components（GCPC）](https://pypi.org/project/google-cloud-pipeline-components/)、[Kubeflow Pipelines（KFP）](https://pypi.org/project/kfp/)以及各种Vertex AI服务，如[Vertex Pipelines](https://cloud.google.com/vertex-ai/docs/pipelines/introduction)、[Vertex Tensorboard](https://cloud.google.com/vertex-ai/docs/experiments/tensorboard-overview)、[Vertex Training for distributed training](https://cloud.google.com/vertex-ai/docs/training/distributed-training)等加速器，[Vertex Online Prediction](https://cloud.google.com/vertex-ai/docs/predictions/getting-predictions)，以及[Vertex Model Evaluation components](https://cloud.google.com/vertex-ai/docs/pipelines/model-evaluation-component)，构建了一个端到端文本分类流水线，该流水线根据新闻文章的标题和简短描述确定其类别。该演示旨在向开发人员展示如何使用KFP和Vertex Pipelines构建端到端流水线，以对其自己的文本数据进行分类。

### 目标

在本教程中，您将构建一个端到端的文本分类管道，用于分类新闻标题和描述。您将使用KFP、Vertex AI和Vertex Pipelines生成一个经过管理且高度可扩展的解决方案。

文本分类管道包括以下步骤：

- 将数据分割为训练集和验证集
- 调整预训练的 [BERT](https://www.tensorflow.org/text/tutorials/classify_text_with_bert) 模型
- 将您的模型上传至Vertex
- 将您的模型部署到一个Vertex端点
- 执行模型评估

### 数据集

该演示使用了一个 Kaggle [新闻分类数据集](https://www.kaggle.com/datasets/rmisra/news-category-dataset)，其中包含从2012年至2018年从 HuffPost 获取的约20万条新闻标题。它位于公共示例云存储桶中，路径为 `gs://cloud-samples-data/vertex-ai/community-content/datasets/news/news_category_data.json`。

### 成本

本教程使用 Google Cloud 的计费组件：

* Vertex AI
* Cloud Storage

请了解[Vertex AI
定价](https://cloud.google.com/vertex-ai/pricing)和[Cloud Storage
定价](https://cloud.google.com/storage/pricing)，并使用[Pricing
Calculator](https://cloud.google.com/products/calculator/)
根据您的预期使用量生成成本估算。

### 设置您的本地开发环境

**如果您正在使用Colab或Vertex AI Workbench笔记本**，您的环境已经满足运行此笔记本的所有要求。请跳转到下面的“安装额外软件包”部分。

否则，请确保您的环境符合本笔记本的要求。
您需要以下内容：

* Google Cloud SDK
* Git
* Python 3
* virtualenv
* 在使用Python 3的虚拟环境中运行Jupyter笔记本

Google Cloud指南中提供了[设置Python开发环境的详细说明](https://cloud.google.com/python/setup)，以及[Jupyter安装指南](https://jupyter.org/install)，
提供了满足这些要求的详细说明。以下步骤提供了一套简化的指令：

1. [安装并初始化Cloud SDK。](https://cloud.google.com/sdk/docs/)

2. [安装Python 3。](https://cloud.google.com/python/setup#installing_python)

3. [安装virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv) 并创建一个使用Python 3的虚拟环境。激活虚拟环境。

4. 要安装Jupyter，请在终端窗口的命令行中运行 `pip3 install jupyter`。

5. 要启动Jupyter，请在终端窗口的命令行中运行 `jupyter notebook`。

6. 在Jupyter Notebook Dashboard中打开这个笔记本。

### 安装额外的包

注意：此笔记依赖于较旧版本的 Google Cloud Pipeline Components library (GCPC) 中的组件。

请安装以下包以执行此笔记。

In [ ]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

In [ ]:
! pip3 install --upgrade pip google-cloud-aiplatform 'google-cloud-pipeline-components<2' kfp tensorflow tensorboard numpy {USER_FLAG} -q

### 重新启动内核

重新启动您的笔记本内核，以确保可以找到所有新安装的软件包。

In [ ]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

在你开始之前

### 设置您的Google Cloud项目

**无论您的笔记本环境如何，都需要执行以下步骤。**

1. [选择或创建一个Google Cloud项目](https://console.cloud.google.com/cloud-resource-manager)。当您首次创建帐户时，您将获得$300免费信用额度用于计算/存储成本。

1. [确保您的项目已启用计费](https://cloud.google.com/billing/docs/how-to/modify-project)。

1. [启用Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,storage.googleapis.com)。

1. 如果您在本地运行此笔记本，您需要安装[Cloud SDK](https://cloud.google.com/sdk)。

1. 在下面的单元格中输入您的项目ID。然后运行单元格，确保Cloud SDK在本笔记本中的所有命令中使用正确的项目。

**注意**：Jupyter会将以`!`为前缀的行视为shell命令，并将以`$`为前缀的Python变量插入这些命令中。

设置您的项目ID

在这里设置您的项目ID（*不是*项目编号）。 如果您不知道您的项目ID，请尝试以下操作：

* 运行`gcloud config list`。
* 运行`gcloud projects list`。
* 查看支持页面：查找项目ID

In [ ]:
PROJECT_ID = "[your-project-id-here]"  # @param {type:"string"}
!gcloud config set project $PROJECT_ID

#### 区域

您还可以更改`REGION`变量，该变量用于笔记本其余部分的操作。以下是支持Vertex AI的区域。我们建议您选择最靠近您的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太：`asia-east1`

您可能不会在Vertex AI上使用多区域存储桶进行训练。并非所有区域都支持所有Vertex AI服务。

了解更多关于[Vertex AI区域](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "[your-region]"  # @param {type: "string"}

if REGION == "[your-region]":
    REGION = "us-central1"

In [ ]:
import random
import string


# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

### 验证您的谷歌云账户

**如果您正在使用Vertex AI工作台笔记本**，您的环境已经经过验证。跳过此步骤。

**如果您正在使用Colab**，运行下面的单元格，并按照提示进行身份验证以通过oAuth进行验证您的账户。

**否则**，请按照以下步骤操作：

1. 在云控制台中，转到[**创建服务账户密钥**页面](https://console.cloud.google.com/apis/credentials/serviceaccountkey)。

2. 点击**创建服务账户**。

3. 在**服务账户名称**字段中输入一个名称，并点击**创建**。

4. 在**授予此服务账户对项目的访问权限**部分，点击**角色**下拉列表。在过滤框中输入“Vertex AI”，并选择**Vertex AI管理员**。在过滤框中输入“存储对象管理员”，并选择**存储对象管理员**。

5. 点击*创建*。一个包含您密钥的JSON文件将下载到您的本地环境中。

6. 在下面的单元格中，将您的服务账户密钥的路径作为`GOOGLE_APPLICATION_CREDENTIALS`变量输入，并运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = "google.colab" in sys.modules
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建一个云存储桶

**无论你使用的是哪个笔记本环境，以下步骤都是必需的。**

在本教程中，一个云存储桶保存了用于训练模型的新闻类别数据集文件。Vertex AI 还会将由预处理组件生成的拆分训练和验证数据集等工件保存在同一个存储桶中。使用这个模型工件，你可以创建一个 Vertex AI 模型和端点，以便提供在线预测。

请在下面设置你的云存储桶的名称。它在所有的云存储桶中必须是唯一的。

In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "aip-" + UUID
    BUCKET_URI = f"gs://{BUCKET_NAME}"

只有当您的存储桶尚不存在时：运行以下单元格来创建您的云存储桶。

In [ ]:
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

最后，通过检查其内容来验证对云存储存储桶的访问。

In [ ]:
! gsutil ls -al $BUCKET_URI

设置服务账号和权限
将使用服务账号来创建自定义培训作业。如果您不想使用项目的计算引擎服务账号，请将 SERVICE_ACCOUNT 设置为另一个服务账号 ID。您可以按照[说明](https://cloud.google.com/iam/docs/creating-managing-service-accounts#creating)创建服务账号。

In [ ]:
SERVICE_ACCOUNT = "[your-service-account]"  # @param {type:"string"}

In [ ]:
if (
    SERVICE_ACCOUNT == ""
    or SERVICE_ACCOUNT is None
    or SERVICE_ACCOUNT == "[your-service-account]"
):
    # Get your service account from gcloud
    if not IS_COLAB:
        shell_output = !gcloud auth list 2>/dev/null
        SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()

    else:  # IS_COLAB:
        shell_output = ! gcloud projects describe  $PROJECT_ID
        project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
        SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"

    print("Service Account:", SERVICE_ACCOUNT)

设置服务帐户访问权限

运行以下命令，将读写管道工件的访问权限授予您的服务帐户，该服务帐户位于您在上一步创建的存储桶中。 您只需对每个服务帐户运行此步骤一次。

In [ ]:
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectCreator $BUCKET_URI

! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectViewer $BUCKET_URI

### 导入库

In [ ]:
from google.cloud import aiplatform
from kfp import components
from kfp.v2 import compiler, dsl
from kfp.v2.dsl import InputPath, component

In [ ]:
# Model evaluation components
from google_cloud_pipeline_components.experimental.evaluation import \
    GetVertexModelOp as get_vertex_model_op
from google_cloud_pipeline_components.experimental.evaluation import \
    ModelEvaluationClassificationOp as evaluation_classification_op
from google_cloud_pipeline_components.experimental.evaluation import \
    ModelImportEvaluationOp as model_import_evaluation_op
from google_cloud_pipeline_components.experimental.evaluation import \
    TargetFieldDataRemoverOp as target_field_data_remover_op
# Text Classification components
from google_cloud_pipeline_components.experimental.sklearn import \
    SklearnTrainTestSplitJsonlOp as train_test_split_op
from google_cloud_pipeline_components.experimental.text_classification import \
    TextClassificationTrainingOp as text_classification_training_op
from google_cloud_pipeline_components.v1.batch_predict_job import \
    ModelBatchPredictOp as batch_prediction_op

### 填写以下必填配置

这个管道接受一个 JSONL 数据集，其中每个 JSON 对象示例都有两个必需的键：`text` 和 `label`。`text` 键应映射到样本的文本数据，而 `label` 键应映射到其分类类别。

以下是在此演示中使用的数据集中的 JSON 对象示例：

{
**"label"**:"CRIME",
**"text"**:"There Were 2 Mass Shootings In Texas Last Week, But Only 1 On TV"
}

In [ ]:
BASE_OUTPUT_DIR = f"gs://{BUCKET_NAME}"

SAMPLE_DATA_URI = "gs://cloud-samples-data/vertex-ai/community-content/datasets/news/news_category_data.json"
TRAINING_DATA_URI = (
    f"{BASE_OUTPUT_DIR}/data/news_category_data.json"  # @param {type:"string"}
)

# The GCS directory for keeping staging files for model evaluation.
ROOT_DIR = f"{BASE_OUTPUT_DIR}/root"  # @param {type:"string"}

将示例数据复制到TRAINING_DATA_URI。

In [ ]:
! gsutil cp {SAMPLE_DATA_URI} {TRAINING_DATA_URI}

`CLASS_NAMES` 应该是一个列表，包含一个文本样本可以分类为的所有类别。

在这个演示中，有 41 个题材类别（如下所示）可以将一个标题分类为。

In [ ]:
CLASS_NAMES = [
    "GOOD NEWS",
    "STYLE",
    "STYLE & BEAUTY",
    "ARTS",
    "IMPACT",
    "WEIRD NEWS",
    "FIFTY",
    "ENTERTAINMENT",
    "ARTS & CULTURE",
    "HEALTHY LIVING",
    "WEDDINGS",
    "PARENTING",
    "BLACK VOICES",
    "GREEN",
    "RELIGION",
    "POLITICS",
    "PARENTS",
    "BUSINESS",
    "DIVORCE",
    "WELLNESS",
    "FOOD & DRINK",
    "THE WORLDPOST",
    "MEDIA",
    "COLLEGE",
    "WOMEN",
    "TASTE",
    "WORLDPOST",
    "TRAVEL",
    "CULTURE & ARTS",
    "SPORTS",
    "CRIME",
    "QUEER VOICES",
    "TECH",
    "COMEDY",
    "MONEY",
    "WORLD NEWS",
    "LATINO VOICES",
    "SCIENCE",
    "EDUCATION",
    "HOME & LIVING",
    "ENVIRONMENT",
]

### 加载组件
KFP SDK提供了各种方法来[加载组件](https://www.kubeflow.org/docs/components/pipelines/sdk/component-development/#using-your-component-in-a-pipeline)以在流水线中使用。在此演示中，我们将加载五个组件。

此流水线由以下组件组成：

- **train_test_split_jsonl_with_sklearn** - 将数据分割为训练和验证数据集。
- **train_tensorflow_text_classification_model** - 创建经过训练的文本分类TensorFlow模型。
- **upload_Tensorflow_model_to_Google_Cloud_Vertex_AI** - 将TensorFlow模型转换为Vertex模型并上传到Vertex。
- **deploy_model_to_endpoint** - 将Vertex模型部署到用于在线预测的端点。
- **get_gcs_uris_from_jsonl_artifact** - 基于Python函数的操作，将数据资源转换为模型评估组件可接受的格式。
- **target_field_data_remover** - 删除验证数据中的目标（标签）字段，以进行下游Vertex批处理预测。
- **model_batch_predict** - 提交批量预测作业。
- **model_evaluation_classification** - 计算并导出评估指标。
- **model_evaluation_import** - 导入模型评估指标结果。

对于已由GCPC提供并发布的组件，请使用`load_component_from_url`。

In [ ]:
upload_tensorflow_model_to_vertex_op = components.load_component_from_url(
    "https://raw.githubusercontent.com/Ark-kun/pipeline_components/c6a8b67d1ada2cc17665c99ff6b410df588bee28/components/google-cloud/Vertex_AI/Models/Upload_Tensorflow_model/workaround_for_buggy_KFPv2_compiler/component.yaml"
)
deploy_model_to_endpoint_op = components.load_component_from_url(
    "https://raw.githubusercontent.com/Ark-kun/pipeline_components/27a5ea25e849c9e8c0cb6ed65518bc3ece259aaf/components/google-cloud/Vertex_AI/Models/Deploy_to_endpoint/workaround_for_buggy_KFPv2_compiler/component.yaml"
)

将验证数据（具有"JSONLines"注释的Artifact）转换为可由下游的"target_field_data_remover_op"和"model_evaluation_op"接收的输出参数gcs_source_uris（类型：Sequence[str]）。

In [ ]:
@component(
    base_image="python:3.9",
)
def get_gcs_uris_from_jsonl_artifact(input_jsonl: InputPath("JSONLines")) -> list:
    return ["gs://" + input_jsonl[5:]]

### 建立一个流水线
以下流水线代码链接加载组件的输入和输出。生成的流水线执行以下步骤：
- 将数据分成训练集和测试集。
- 训练新的文本分类模型。
- 将模型上传到 Vertex AI Model Registry。
- 使用测试数据执行批量预测。
- 使用上述批量预测评估模型的性能。
- 将评估指标导入模型。

In [ ]:
@dsl.pipeline(name="text-classification-pipeline")
def text_pipeline(
    project: str,
    training_data_uri: str,
    class_names: list,
    root_dir: str,
    target_field_name: str,
    batch_predict_display_name: str,
    batch_predict_instances_format: str = "jsonl",
    batch_predictions_format: str = "jsonl",
    model_name: str = "small_bert/bert_en_uncased_L-2_H-128_A-2",
    validation_split: float = 0.2,
    batch_size: int = 256,
    learning_rate: float = 3e-4,
    num_epochs: int = 5,
    random_seed: int = 0,
) -> None:
    """End-to-end text classification pipeline.

    Args:
    project: Required. GCP project ID.
    training_data_uri: Required. Data in JSON lines format.
    class_names: Required. List of categories (string) for classification.
    root_dir: Required. The GCS directory for keeping staging files for model evaluation.
    target_field_name: Required. The name of the features target field in the predictions file (e.g. 'label').
    batch_predict_instances_format: The file format for the ground truth files.
    batch_predictions_format: The file format for the batch prediction results.
    batch_predict_display_name: Required. The user-defined name of this BatchPredictionJob.
    model_name: Optional. Name of pre-trained BERT model to be used.
                Default: "small_bert/bert_en_uncased_L-2_H-128_A-2"
    validation_split: Optional. Fraction of data that will make up validation dataset.
                      Default: 0.2
    batch_size: Optional. Batch size
                Default: 256
    learning_rate: Optional. Learning rate
                   Default: 3e-4
    num_epochs: Optional. Number of epochs
                Default: 10
    random_seed: Optional. Random seed
                 Default: 0
    """

    text_data_preprocess_task = train_test_split_op(
        input_data_path=training_data_uri,
    )

    training_data = text_data_preprocess_task.outputs["training_data_path"]

    validation_data = text_data_preprocess_task.outputs["validation_data_path"]

    # Set CPU, memory, and GPU configuration settings for this step (https://cloud.google.com/vertex-ai/docs/pipelines/machine-types)
    model = (
        text_classification_training_op(
            preprocessed_training_data_path=training_data,
            preprocessed_validation_data_path=validation_data,
            model_name=model_name,
            class_names=class_names,
            batch_size=batch_size,
            learning_rate=learning_rate,
            num_epochs=num_epochs,
            random_seed=random_seed,
        )
    ).add_node_selector_constraint(
        "cloud.google.com/gke-accelerator", "NVIDIA_TESLA_A100"
    )  # Note that A100 is available on us-central1

    vertex_model_name = upload_tensorflow_model_to_vertex_op(
        model=model.outputs["trained_model_path"],
    ).outputs["model_name"]

    # Model evaluation
    # Need a component to convert Artifact('JsonLinesDataset') to JsonArray
    validation_data_uris = get_gcs_uris_from_jsonl_artifact(validation_data).output

    evaluation_data_for_batch_predict = target_field_data_remover_op(
        project=project,
        root_dir=root_dir,
        target_field_name=target_field_name,
        gcs_source_uris=validation_data_uris,
    ).outputs["gcs_output_directory"]

    vertex_model = get_vertex_model_op(
        model_resource_name=vertex_model_name,
    ).outputs["model"]

    batch_prediction_task = batch_prediction_op(
        project=project,
        model=vertex_model,
        job_display_name=batch_predict_display_name,
        gcs_source_uris=evaluation_data_for_batch_predict,
        gcs_destination_output_uri_prefix=root_dir,
        instances_format=batch_predict_instances_format,
        predictions_format=batch_predictions_format,
        machine_type="n1-standard-32",
        starting_replica_count=5,
        max_replica_count=10,
    )

    # Run the evaluation based on prediction type
    eval_task = evaluation_classification_op(
        project=project,
        root_dir=root_dir,
        ground_truth_gcs_source=validation_data_uris,
        target_field_name=target_field_name,
        prediction_score_column="prediction",
        prediction_label_column="",
        class_labels=class_names,
        ground_truth_format=batch_predict_instances_format,
        predictions_format=batch_predictions_format,
        predictions_gcs_source=batch_prediction_task.outputs["gcs_output_directory"],
    )
    # Import the model evaluations to the Vertex AI model
    model_import_evaluation_op(
        classification_metrics=eval_task.outputs["evaluation_metrics"],
        model=vertex_model,
        dataset_type="jsonl",
    )

    # For online predictions
    _ = deploy_model_to_endpoint_op(
        model_name=vertex_model_name,
    ).outputs["endpoint_name"]


pipeline_func = text_pipeline

运行管线

以下的代码块从上面的pipeline函数创建一个流水线运行，并提交到Vertex AI平台。您可以在[Vertex ML Metadata (MLMD)](https://cloud.google.com/vertex-ai/docs/ml-metadata/introduction) 中查看该流水线的工件，在运行下一个代码块时会输出链接。 

只需要修改这个流水线的参数以适应您特定的用例。在`parameter_values`字典中指定所需的流水线参数和任何可选的参数。

In [ ]:
compiler.Compiler().compile(
    pipeline_func=pipeline_func,
    package_path="text_classification_pipeline.json",
)

In [ ]:
PIPELINE_DISPLAY_NAME = f"text-classification-train-evaluate-{UUID}"  # "[your-pipeline-display-name]"  # @param {type:"string"}

BATCH_PREDICTION_DISPLAY_NAME = f"batch-prediction-on-pipelines-model-{UUID}"

parameters = {
    "project": PROJECT_ID,
    "training_data_uri": TRAINING_DATA_URI,
    "class_names": CLASS_NAMES,
    "num_epochs": 5,
    "root_dir": ROOT_DIR,
    "target_field_name": "label",
    "batch_predict_display_name": BATCH_PREDICTION_DISPLAY_NAME,
}

In [ ]:
job = aiplatform.PipelineJob(
    display_name=PIPELINE_DISPLAY_NAME,
    template_path="text_classification_pipeline.json",
    location=REGION,
    enable_caching=True,
    parameter_values=parameters,
    project=PROJECT_ID,
)

job.submit(service_account=SERVICE_ACCOUNT)

## 进行预测

一旦您的模型部署到一个端点，它可以使用UI或KFP SDK进行预测。

要使用UI，请观看这个简短的[教程](https://screencast.googleplex.com/cast/NDY5Nzk3NzUzNzk1Mzc5MnxiNDMxZTFkNi1lNg)或按照[这里](https://cloud.google.com/vertex-ai/docs/predictions/online-predictions-custom-models)所示的步骤进行操作。

要使用KFP SDK 进行预测，需要知道部署模型的端点。以下代码从PipelineJob中提取`ENDPOINT_ID`。

In [ ]:
task_id = "deploy-model-to-endpoint-for-google-cloud-vertex-ai-model"
deploy_task_detail = [
    task_details
    for task_details in job.task_details
    if task_details.task_name == task_id
][0]
ENDPOINT_ID = deploy_task_detail.execution.metadata["output:endpoint_name"]

每个请求必须是一个带有`text`键的JSON对象。`instances`应该是一个包含所有请求的列表。

In [ ]:
instances = [
    {
        "text": "Irish Voters Set To Liberalize Abortion Laws In Landslide, Exit Poll Signals Vote counting will begin Saturday."
    }
]

以上順序結構化的要求列表中的預測清單。每個預測都是一個向量，其中包含 `class_names` 中每個類別作為正確標籤的概率。概率與 `class_names` 中每個類別的順序對應。

In [ ]:
endpoint = aiplatform.Endpoint(ENDPOINT_ID)
prediction = endpoint.predict(instances=instances)
print(prediction)

清理

要清理此项目中使用的所有Google Cloud资源，您可以[删除用于教程的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在本教程中创建的各个资源。

In [ ]:
# Delete GCS bucket.
! gsutil -m rm -r {BUCKET_URI}

# Delete endpoint resource.
! gcloud ai endpoints delete $ENDPOINT_ID --quiet --region $REGION